## Import statements

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn
#px.extension('plotly')
import hvplot.pandas

import matplotlib.pyplot as plt

import os
from pathlib import Path
from dotenv import load_dotenv
import holoviews as hv
import requests
import master_functions_cross_validate as fn
import time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

window_size = 20



%config IPCompleter.use_jedi = False

%config IPCompleter.greedy=True


## Input Ticker symbol

In [3]:
tickers_list = []

number_of_tickers = int(input("Enter the number of ticker symbols to analyze: "))

list_iterator = 0

while list_iterator<number_of_tickers:
    print(f"Value of iterator - {list_iterator}")
    ticker_symbol = input("Please provide a stock ticker symbol to analyze: ")
    
    if fn.validate_ticker(ticker_symbol):
        ticker_data = fn.get_ticker_data(ticker_symbol)
        
        if ticker_data == False:
            print("Please enter a valid ticker symbol")
        else:
            file_name_ticker_data = f'Data/{ticker_symbol}_data.csv'
            with open(file_name_ticker_data, 'w') as file:
                file.write(ticker_data)
                
            tickers_list.append(ticker_symbol)
            list_iterator = list_iterator + 1
    else:
        print("Please enter a valid ticker symbol")

Enter the number of ticker symbols to analyze:  1


Value of iterator - 0


Please provide a stock ticker symbol to analyze:  AAPL


In [4]:
#tickers_list = ['AAPL']

ticker_data_dict = {} # dictionary to hold ticker dataframes
ticker_data_predicted = {} # dictionary to hold predicted ticker dataframes
ticker_data_forecasts = {} # dictionary to hold forcast dataframes


# Loop through the ticker list and build the Prediction model for each ticker

In [5]:
for ticker in tickers_list:
    ticker_df = fn.create_df(ticker)
    ticker_data_dict[ticker] = ticker_df
    ticker_predicted_data = fn.model_predictions(ticker_df, window_size, ticker)
    ticker_data_predicted[ticker] = ticker_predicted_data

For window size=20 and ticker - AAPL the loss is 1.5754135847091675
5/5 [==============================] - 1s 5ms/step
cross Validation scores:n [0.03333333 0.03333333 0.03333333 0.         0.03333333 0.
 0.         0.         0.03333333 0.         0.         0.
 0.         0.03333333 0.         0.         0.         0.
 0.03333333 0.03333333]
Average Cross Validation score :0.013333333333333332


In [14]:
for key in ticker_data_dict.keys():
    ticker_df = ticker_data_dict[key].sort_index(ascending = True)
    ticker_forecasts = fn.model_forecasts(ticker_df, window_size, key)
    #ticker_forecasts = ticker_forecasts.drop(columns=["Actual"])
    ticker_data_forecasts[key] = ticker_forecasts

1/1 [==============================] - 1s 538ms/step


In [6]:
for key in ticker_data_predicted.keys():
    ticker_df = ticker_data_predicted[key]
    plot_title = f"{key}"
    print(plot_title.upper())
    display(ticker_df.hvplot(title=plot_title))

AAPL


:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [16]:
for key in ticker_data_forecasts.keys():
    ticker_df = ticker_data_forecasts[key]
    plot_title = f"{key}"
    print(plot_title.upper())
    display(ticker_df.hvplot(title=plot_title))

AAPL


:NdOverlay   [Variable]
   :Curve   [Date]   (value)